In [66]:
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from bs4 import BeautifulSoup
import time
import os
import requests
import pandas as pd
from selenium.webdriver.common.by import By

In [67]:
df_neg = pd.read_csv("Data/PV web_positive_2.csv")
# Menambahkan "https://" di setiap nilai URL
df_neg['URL'] = df_neg['URL'].apply(lambda url: 'https://' + url)
data_url = df_neg['URL'].to_numpy()
df_neg.shape

(192, 2)

In [68]:
# BUKA CHROME
chrome_options = Options()
chrome_options.add_argument('--no-sandbox')
chrome_options.add_argument('--disable-dev-shm-usage')
chrome_options.add_argument('--disable-gpu')

driver = webdriver.Chrome(options=chrome_options)

In [69]:
import re

# Fungsi untuk menghapus pengulangan kata
def remove_duplicate_words(text):
    words = text.split()
    unique_words = []
    for word in words:
        if word not in unique_words:
            unique_words.append(word)
    return ' '.join(unique_words)

def add_spasi(text):
    hasil = re.sub(r'(?<=[a-z])([A-Z])', r' \1', text)
    return hasil 

In [70]:
data_hasil_url = []
data_hasil_text = []
for i in range(len(data_url)) :
    url = data_url[i]
    # Waktu awal
    start_time = time.time()
    try :
        driver.get(url)
        time.sleep(5)
        
        print("masuk : "+url)
        # Printing the whole body text
        teks = driver.find_element(By.XPATH, "/html/body").text
        
        # clean dikit
        kalimat = add_spasi(teks)
        kalimat = remove_duplicate_words(kalimat.lower())

        data_hasil_url.append(url)
        data_hasil_text.append(kalimat)
        
        print("____________________________")
        print("Hasil Pengambilan Url: ", url)
        print("Hasil Pengambilan Text: ", kalimat)
        print("____________________________")
        
        # Memeriksa apakah sudah lebih dari 15 detik
        current_time = time.time()
        elapsed_time = current_time - start_time
        
    except Exception as e :
        print("Error : ",url)
        continue

masuk : https://wikipedia.org
____________________________
Hasil Pengambilan Url:  https://wikipedia.org
Hasil Pengambilan Text:  wikipedia the free encyclopedia english 6 715 000+ articles 日本語 1 387 記事 español 892 artículos русский 938 статей deutsch 2 836 artikel français 553 italiano 826 voci 中文 377 条目 / 條目 português 109 artigos العربية 217 مقالة search en afrikaans polski asturianu azərbaycanca български বাংলা беларуская català čeština cymraeg dansk eesti ελληνικά esperanto euskara فارسی galego 한국어 हिन्दी hrvatski bahasa indonesia עברית ქართული latina latviešu lietuvių magyar македонски مصرى melayu bahaso minangkabau nederlands norsk (bokmål) (nynorsk) нохчийн oʻzbekcha ўзбекча қазақша qazaqşa قازاقشا română simple sinugboanong binisaya slovenčina slovenščina српски srpski srpskohrvatski српскохрватски suomi svenska தமிழ் татарча tatarça ภาษาไทย тоҷикӣ تۆرکجه türkçe українська اردو tiếng việt winaray հայերեն မြန်မာဘာသာ read in your language is hosted by wikimedia foundation, a non-

In [71]:
df = pd.DataFrame({"url" : data_hasil_url, "text" : data_hasil_text})
df.shape

(174, 2)

In [72]:
# Menghapus baris dengan nilai kosong, None, atau hanya berisi string kosong
df = df[df['text'].notna() & (df['text'] != '') & (df['text'].str.strip() != '')]
df.shape

(173, 2)

In [74]:
df.to_csv("Hasil Scrap/data_hasil_scrap_positive.csv", index=False)